### Optimizing Temporary Housing Deployment to Reduce U.S. Homelessness 
Nauman Sohani, Yannan Tuo, Charlie Nitschelm

In [1]:
# Load packages
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# Generate distance matrix
# %run ./optimization/coc-distance.ipynb
# %whos

# print(coc_distance_matrix.head())

In [3]:
# Set up Gurobi environment
env = gp.Env(empty=True)
env.setParam('OutputFlag', 0)
env.start()

# Initialize the model
m = gp.Model(env=env)

In [4]:
# High level variables + assumptions
# Total budget allocated for uplift units
TotalBudget =  50000000 # 130000000 # 50000 # 10000000 # budget: 85281049.54872459

# Uplift unit specification assumptions
# Uplift Factory Spec 
from pathlib import Path
directory = Path.cwd()
FactoryCost = pd.read_csv(directory / 'factory_estimate.csv')
FactoryCost.set_index('CoC_Number', inplace=True)

# FactoryCost = 10000 

# Uplift THU Spec 
THUCost = 5_000

# Uplift Shipping Spec 
THUShippingBase = 100
THUShippingPerMile = 1

# Uplift Operational Spec 
# THU_Factory_Limit = 999999999999999  # 15000 #1500
THU_Factory_Limit = 5_000  # 15000 #1500
# THU_to_Factory_Max_Distance = 50

# Big M
M = 999999999999999999999

# Given no factory production limit, how much budget is needed for it to make sense to build second factory
# vary over budget


In [5]:
# City distance data
coc_distance_matrix = pd.read_csv(directory /'coc_distance_matrix.csv')
# coc_distance_matrix.set_index('CoC_Number', inplace=True)
# print(coc_distance_matrix.head())
# city_distance = {
#                  "SF": {"SF": 0, "Boston": 3000, "NYC": 2900}, 
#                  "Boston": {"SF": 3000, "Boston": 0, "NYC": 215},
#                  "NYC": {"SF": 2900, "Boston": 215, "NYC": 0}
#                  }

print(coc_distance_matrix.head())

       AZ_500      AZ_501      AZ_502      CA_500      CA_501      CA_502  \
0    0.000000  100.377884   95.797626  641.657111  667.456178  535.237009   
1  100.377884    0.000000  172.913722  638.231594  653.762249  531.295315   
2   95.797626  172.913722    0.000000  728.159821  757.382496  622.740097   
3  641.657111  638.231594  728.159821    0.000000   69.615583  107.118344   
4  667.456178  653.762249  757.382496   69.615583    0.000000  140.210217   

       CA_503      CA_504      CA_505      CA_506  ...       TX_611  \
0  559.778585  655.119883  695.660884  727.027926  ...   913.840076   
1  564.563350  655.039749  691.616312  702.218126  ...   861.619424   
2  643.580939  740.194331  782.071906  819.744306  ...   881.790039   
3   92.088930   26.308731   54.003774  163.041002  ...  1461.616336   
4  149.227808   89.195966   72.962575   94.250202  ...  1456.684841   

        TX_624       TX_700       TX_701      UT_500      UT_503      UT_504  \
0   763.895815   866.257526   

In [6]:
# CoC Data/homeless population
CoC_populations = pd.read_csv(directory /'Coc_populations.csv')
cocs = CoC_populations['CoC_Number'].tolist()
CoC_populations.set_index('CoC_Number', inplace=True)
numCoCs = len(CoC_populations)
print(cocs)
# CoCPopulations = {"SF": 8300, "Boston": 6000, "NYC": 350000}
# CoCPopulations = {"SF": 9000, "Boston": 6000, "NYC": 15000}
# CoCPopulations = {"SF": 5000, "Boston": 5000, "NYC": 5000}

# Number of cities
NumCities = numCoCs

# Number of factory locations available (equivalent to cities)
# todo: rename var
NumFactories = numCoCs

# ordered dict for version < 3.7; assumed same order in version >= 3.7
indices = range(len(CoC_populations))
# CoCs = CoC_populations['CoC_Number'].values
indexToCityDict = dict(zip(indices, cocs))
def indexToCity(index):
    return indexToCityDict[index]

# print(indexToCityDict)

print(NumCities)
print(NumFactories)
print(len(FactoryCost))

['AZ_500', 'AZ_501', 'AZ_502', 'CA_500', 'CA_501', 'CA_502', 'CA_503', 'CA_504', 'CA_505', 'CA_506', 'CA_507', 'CA_508', 'CA_509', 'CA_510', 'CA_511', 'CA_512', 'CA_513', 'CA_514', 'CA_515', 'CA_516', 'CA_517', 'CA_518', 'CA_519', 'CA_520', 'CA_521', 'CA_522', 'CA_523', 'CA_524', 'CA_525', 'CA_526', 'CA_527', 'CA_529', 'CA_530', 'CA_531', 'CA_600', 'CA_601', 'CA_602', 'CA_603', 'CA_604', 'CA_606', 'CA_607', 'CA_608', 'CA_609', 'CA_611', 'CA_612', 'CA_613', 'CA_614', 'CT_503', 'CT_505', 'MA_500', 'MA_502', 'MA_503', 'MA_504', 'MA_505', 'MA_506', 'MA_507', 'MA_509', 'MA_511', 'MA_515', 'MA_516', 'ME_500', 'NH_500', 'NH_501', 'NH_502', 'NJ_500', 'NJ_501', 'NJ_502', 'NJ_503', 'NJ_504', 'NJ_506', 'NJ_507', 'NJ_508', 'NJ_509', 'NJ_510', 'NJ_511', 'NJ_512', 'NJ_513', 'NJ_514', 'NJ_515', 'NJ_516', 'NM_500', 'NM_501', 'NV_500', 'NV_501', 'NV_502', 'NY_500', 'NY_501', 'NY_503', 'NY_505', 'NY_507', 'NY_508', 'NY_510', 'NY_511', 'NY_512', 'NY_513', 'NY_514', 'NY_518', 'NY_519', 'NY_520', 'NY_522',

In [7]:
# Decision variables
# How many homes to place per city
THU = m.addVars((t for t in range(0, NumCities)), lb=0, name="THUQuantityPerCity")

# Where to place factories (also # of factories)
Factory = m.addVars((t for t in range(0, NumFactories)), vtype=GRB.BINARY, name="FactoryLocations")

# How many THUs shipped from a given factory
THUShippedFromFactory = m.addMVar((NumCities, NumFactories), name="ClosestFactory", lb=0)
# City = m.addVars(((i, j) for i in range(0, NumCities) for j in range(0, NumFactories)), lb=0, ub = 1, name="ClosestFactory")


In [8]:
# Constraints
# Budget constraint
# sum cost of all the THUs + 
# sum cost of all the factories + 
# sum cost of transportation of THUs from all the factories = # produced in city * closest factory * distance to factory * shipping cost per mile
budgetConstr = m.addConstr((sum(THU[cityIndex]*THUCost for cityIndex in range(0, NumCities)) + 
                              sum(FactoryCost.iloc[factory]['microhome_cost']*Factory[factory] for factory in range(NumFactories)) +
                              sum(THUShippedFromFactory[(cityIndex, factory)]*coc_distance_matrix.iloc[cityIndex, factory]*THUShippingPerMile 
                              for cityIndex in range(NumCities) for factory in range(NumFactories))) <= TotalBudget, name='BudgetConstr')

# Unhoused population (allocate no more than # unhoused)
popConstr = m.addConstrs(THU[cityIndex] <= CoC_populations.iloc[cityIndex] for cityIndex in range(0, NumCities))

# sum of THUs shipped needs to equal THUs in location
THUTotal = m.addConstrs(sum(THUShippedFromFactory[cityIndex][factoryIndex] for factoryIndex in range(NumFactories)) == THU[cityIndex] for cityIndex in range(NumCities))

# enforce no THU production if factory not selected
THUProdatFactory = m.addConstrs(THUShippedFromFactory[cityIndex][factoryIndex] <= M*Factory[factoryIndex] for cityIndex in range(NumCities) for factoryIndex in range(NumFactories))
FactoryLimit = m.addConstrs(sum(THUShippedFromFactory[cityIndex][factoryIndex] for cityIndex in range(NumCities)) <= THU_Factory_Limit for factoryIndex in range(NumFactories))

# Todo: max and min # THUs per factory

# There must be at least one factory in total
factoryConstr = m.addConstr(sum(Factory[factory] for factory in range(NumFactories)) >= 1)


/var/folders/hm/hb18k8k54xd7nbc0n2gg1yh80000gn/T/ipykernel_61152/3849584006.py:12: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  popConstr = m.addConstrs(THU[cityIndex] <= CoC_populations.iloc[cityIndex] for cityIndex in range(0, NumCities))


In [9]:
# Objective function
# Maximize number of housing units provided for unhoused populations (ie; minimize unhoused individiuals)
m.setObjective(sum(THU[city] for city in range(0, NumCities)), gp.GRB.MAXIMIZE)


In [10]:
# Update and write the model
m.update() # Update model parameters
# m.write("uplift.lp") # Write model to file

# Solve
m.optimize()

# Check model is not infeasible
if m.status == GRB.INFEASIBLE:
    print("model is infeasible")
print("\nObjective value: ", "%.2f" % m.getAttr("ObjVal"))



Objective value:  8023.98


In [11]:
# Print solution
print("THUs:")
for i in THU:
    if THU[i].x > 0:
        print('%s: %g' % (indexToCity(i), THU[i].x))

print("Factories:")
for i in Factory:
    if Factory[i].x > 0:
        print('%s: %g' % (indexToCity(i), Factory[i].x))

print("Shipping from factories:")
for i in range(NumCities):
    for j in range(NumFactories):
        if THUShippedFromFactory[i][j].x> 0:
            print('source city: %s, factory city: %s, value: %g' % (indexToCity(i), indexToCity(j), THUShippedFromFactory[i][j].x))

THUExpense = sum(THU[cityIndex].x*THUCost for cityIndex in range(0, NumCities))
FactoryExpense = sum(FactoryCost.iloc[factory]['microhome_cost']*Factory[factory].x for factory in range(NumFactories))
shipCost = sum(THUShippedFromFactory[cityIndex][factory].x*coc_distance_matrix.iloc[cityIndex, factory]*THUShippingPerMile for cityIndex in range(NumCities) for factory in range(NumFactories))
print("Expected max budget: " + str(TotalBudget) + " actual budget: " + str(THUExpense + FactoryExpense + shipCost))
print("THU cost: ", THUExpense)
print("Factory cost: " + str(FactoryExpense))
print("THU ship cost: " + str(shipCost))

THUs:
NY_501: 594
NY_503: 889
NY_507: 430
NY_508: 1560
NY_510: 273
NY_600: 3440
PA_509: 837.977
Factories:
NY_501: 1
NY_508: 1
Shipping from factories:
source city: NY_501, factory city: NY_501, value: 594
source city: NY_503, factory city: NY_501, value: 889
source city: NY_507, factory city: NY_501, value: 430
source city: NY_508, factory city: NY_508, value: 1560
source city: NY_510, factory city: NY_501, value: 273
source city: NY_600, factory city: NY_508, value: 3440
source city: PA_509, factory city: NY_501, value: 837.977
Expected max budget: 50000000 actual budget: 50000000.00000001
THU cost:  40119882.51043541
Factory cost: 9551403.064206325
THU ship cost: 328714.4253582749


In [12]:
print(CoC_populations)

            Overall Homeless
CoC_Number                  
AZ_500                2386.0
AZ_501                2209.0
AZ_502                9642.0
CA_500                9903.0
CA_501                7582.0
...                      ...
UT_500                2297.0
UT_503                1165.0
UT_504                 225.0
VT_500                2537.0
VT_501                 758.0

[150 rows x 1 columns]


#### Analysis of optimization results
beep boop